# Assignment 2 : Linear Optimization

Name : Ahmik Virani <br>
Roll Number : ES22BTECH11001

In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import null_space

# Read CSV file
df = pd.read_csv('Testcase.csv', header=None)

# Get total rows and columns
rows, cols = df.shape
m = rows - 2
n = cols - 1

# Extract components
z = df.iloc[0, :-1].dropna().to_numpy(dtype=float)
c = df.iloc[1, :-1].dropna().to_numpy(dtype=float)
A = df.iloc[2:, :-1].to_numpy(dtype=float)
B = df.iloc[2:, -1].to_numpy(dtype=float)

print("n =", n)
print("m =", m)
print("z =", z)
print("c =", c)
print("A =\n", A)
print("B =", B)


n = 2
m = 7
z = [0.5 2.4]
c = [5. 2.]
A =
 [[ 1.  1.]
 [ 3. -1.]
 [ 1.  2.]
 [-1.  0.]
 [ 0. -1.]
 [-1.  1.]
 [ 2.  1.]]
B = [10.  9. 10.  0.  0.  2.  8.]


First Let us handle the assumption that rank(A) = n <br>
To do this, we have seen in class that we can split each variable into sum of two non-negative variables (eg. Split x = x1 - x2, such that x1 and x2 both are non-negative)

In [ ]:
# For this, we split each variable of A into 2
# We also need to add additional constraints that each of the new variable is >= 0 (or negative of that variable is <= 0)
# Which means we need a total of 2*n columns (one for each variable)
# We need m + 2*n rows, (m for the existing constraints, the rest 2*n to ensure each variable >= 0) for A
A_new = np.zeros((m + 2*n,2*n))

# We need m + 2*n size vector for B
B_new = np.zeros(m + 2*n)

for i in range(m):
  for j in range(n):
    # Making each variable of A into 2
    A_new[i,2*j] = A[i,j]
    A_new[i,2*j+1] = -A[i,j]

    # Copying the inital constraints into B
    B_new[i] = B[i]

# Handling constrains that each variable should be non-negative
for i in range(m, m + 2*n, 1):
  A_new[i,i-m] = -1
  B_new[i] = 0

# We also need to ensure that the cost function and the inital given vertex is updated
c_new = np.zeros(2*n)
z_new = np.zeros(2*n)
for i in range(n):
  c_new[2*i] = c[i]
  c_new[2*i+1] = -c[i]

  if(z[i] > 0):
    z_new[2*i] = z[i]
  else:
    z_new[2*i+1] = -z[i]


A = A_new
B = B_new
c = c_new
z = z_new

m = m + 2*n
n = 2*n

print("n =", n)
print("m =", m)
print("z =", z)
print("c =", c)
print("A =\n", A)
print("B =", B)

n = 4
m = 11
z = [0.5 0.  2.4 0. ]
c = [ 5. -5.  2. -2.]
A =
 [[ 1. -1.  1. -1.]
 [ 3. -3. -1.  1.]
 [ 1. -1.  2. -2.]
 [-1.  1.  0. -0.]
 [ 0. -0. -1.  1.]
 [-1.  1.  1. -1.]
 [ 2. -2.  1. -1.]
 [-1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]
B = [10.  9. 10.  0.  0.  2.  8.  0.  0.  0.  0.]


In [ ]:
# Here I define a function to convert the vertex in the new space back to the original space
def convert_vertex(vertex):
  new_vertex = np.zeros(n//2)
  for i in range(n):
    if(i%2 == 0):
      new_vertex[i//2] += vertex[i]
    else:
      new_vertex[i//2] -= vertex[i]
  return new_vertex

Now I am goning to handle degeneracy <br>
Here we only need to update B <br>
We have seen that we need to introduce a variable epsilon close to 0. <br>
If we have chosen a wrong value, we multiply it by half and do the process over again

In [ ]:
# First let me introduce a variable epsilon
epsilon = 1e-1

# Let me also keep track of the original B
B_original = B.copy()

def handle_degeneracy(epsilon):
  B_new = np.zeros(m)
  for i in range(m):
    B_new[i] = B_original[i] + pow(epsilon, i+1)
  return B_new

In [ ]:
# This is a function which takes the initial feasable point to the vertex
def go_to_vertex(z, A, B, m, n):

  path = []
  current_z = z.copy()

  while True:
    # We first need to find the set of tight and untight rows of A
    # Let A1 be the set of tight rows
    # Let A2 be the set of untight rows
    Az = A @ current_z
    A1_indexes = []
    A2_indexes = []
    for i in range(m):
        # For floats, we need to check if they are very close
        if abs(Az[i] - B[i]) < 1e-9:
            A1_indexes.append(i)
        else:
            A2_indexes.append(i)

    A1 = A[A1_indexes, :]
    A2 = A[A2_indexes, :]

    # Ensure that the number of tight rows is not zero
    if len(A1_indexes) == 0:
      rank = 0
    else:
      rank = np.linalg.matrix_rank(A1)

    # If rank of the tight rows is >= n, then this is already a vertex
    if rank >= n:
      if not np.allclose(current_z, z, atol=1e-9):
        path.append(np.round(current_z, 9))
      return (current_z, path)

    # We need to find a direction in the null space of A1
    if rank == 0:
      # We need to move to a random direction
      d = np.random.rand(n)
      d = d / np.linalg.norm(d)
    else:
      d = null_space(A1)[:, 0]

    min_step_size = np.inf

    # If we go towards d or opposite d
    direction = 0

    # First check going towards d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = 1

    # Next check going opposite to d
    for i in A2_indexes:
      Ai_dot_d = np.dot(A[i], -d)
      if Ai_dot_d > 1e-9:
        step_size = (B[i] - Az[i]) / Ai_dot_d
        if step_size < min_step_size:
          min_step_size = step_size
          direction = -1

    current_z += min_step_size * direction * d

In [ ]:
# Since we are handling degeneracy, we need an additional outer loop where we restart if we have picked the wrong epsilom
while(True):
  # We are already given the starting vertex
  current_vertex = z.copy()

  # Ensure to update B every time the outer loop runs
  B = handle_degeneracy(epsilon)

  current_vertex, finding_vertex_path = go_to_vertex(current_vertex, A, B, m, n)

  print(f"Applying Simplex with epsilon = {epsilon}")

  # A variable done to break out of this outer loop
  # If done = 1, we have a finite optimum
  # If done = 2, we have unbounded/infinite optimum
  done = 0

  # Variable to keep track of visited vertices and its corresponding value of objective function
  visited_vertices = []
  visited_values = []

  while(True):
    # We first need to find the set of tight and untight rows of A
    # Let A1 be the set of tight rows
    # Let A2 be the set of untight rows
    Az = A @ current_vertex
    A1_indexes = []
    A2_indexes = []
    for i in range(m):
        # For floats, we need to check if they are very close
        if abs(Az[i] - B[i]) < 1e-9:
            A1_indexes.append(i)
        else:
            A2_indexes.append(i)

    # Check if the current vertex if degenrate
    # Which means that more than n constraints are tight
    if len(A1_indexes) != n:
      for i in range(len(finding_vertex_path)):
        print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i]), 9))
      for i in range(len(visited_vertices)):
        print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
      # We need to update our epsilon
      epsilon /= 2
      print(f"Degeneracy found. Reducing episilon to : {epsilon}")
      print("=============================================================")
      print("\n\n")
      # Then we restart from the outer loop
      break

    A1 = A[A1_indexes, :]
    A2 = A[A2_indexes, :]

    # We know that rank of A1 is n
    # Thus, we know that A1 has as inverse
    A_inv = np.linalg.inv(A1)

    x_original = np.linalg.solve(A1, B_original[A1_indexes])
    visited_vertices.append(convert_vertex(x_original))
    visited_values.append(np.dot(c, x_original))

    # Next we check c.T dot A_inv
    # If even one is negative, then z is not optimum
    all_positive = True

    for row in A_inv.T:
      if np.dot(c.T, row) < -1e-9:
        all_positive = False
        # Move in opposite direction of this
        # Let d be direction of movement
        d = -row

        # We also need the size of movement
        # But we only check along the untight rows, i.e. A2
        step_sizes = []
        for i in A2_indexes:
          Ai_dot_d = np.dot(A[i], d)
          # We move in this direction if it becomes more tight
          if Ai_dot_d > 1e-9:
            step_sizes.append((B[i] - Az[i]) / Ai_dot_d)

        if step_sizes:
            # We choose the minimum step size to ensure we are going to the adjacent vertex
            alpha = min(step_sizes)
            current_vertex += alpha * d
            break
        else:
          # Here, we have not found any direction where rows of A2 become more tight
          # Thus this optimum is unbounded and we report this
          done = 2
          break

    if all_positive is True:
      done = 1

    if done > 0:
      if done == 1:
        for i in range(len(finding_vertex_path)):
          print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i]), 9))
        for i in range(len(visited_vertices)):
          print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
      else:
        for i in range(len(finding_vertex_path)):
          print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i]), 9))
        for i in range(len(visited_vertices)):
          print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
        print("Optimum is unbounded")
      break

  if done > 0:
    break

Applying Simplex with epsilon = 0.1
Trying to reach a vertex before applying Simplex. Moving to:  [-9.00000000e-09  2.00000099e+00]
Vertex Visited: [0. 2.], Objective Function = 4.0
Vertex Visited: [2. 4.], Objective Function = 18.0
Vertex Visited: [3.4 1.2], Objective Function = 19.4


## Reporting the final answer

In [ ]:
if done == 1:
  print("Final Verdict: Optimal Vertex found")
  print("Printing path")
  for i in range(len(finding_vertex_path)):
    print("Trying to reach a vertex before applying Simplex. Moving to: ", np.round(convert_vertex(finding_vertex_path[i]), 9))
  for i in range(len(visited_vertices)):
    print(f"Vertex Visited: {np.round(visited_vertices[i], 9)}, Objective Function = {np.round(visited_values[i], 9)}")
else:
  print("Final Verdict: Optimum is unbounded")

Final Verdict: Optimal Vertex found
Printing path
Trying to reach a vertex before applying Simplex. Moving to:  [-9.00000000e-09  2.00000099e+00]
Vertex Visited: [0. 2.], Objective Function = 4.0
Vertex Visited: [2. 4.], Objective Function = 18.0
Vertex Visited: [3.4 1.2], Objective Function = 19.4


In [ ]:
# Note due to handling degeneracy, we might be seeing the same vertex multiple times
# Please ignore small erros due to python